# Логистическая регрессия. Практика

В этом задании вам предлагается спрогнозировать, купит клиент велосипед или нет, обучив логистическую регрессию.

In [66]:
# подключим библиотеки
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

In [67]:
# считаем данные
data = pd.read_csv('https://raw.githubusercontent.com/evgpat/edu_stepik_practical_ml/main/datasets/bike_buyers_clean.csv')

In [68]:
# выводим первые 5 строк датафрейма
data.head(5)

,ID,Marital Status,Gender,Income,Children,Education,Occupation,Home Owner,Cars,Commute Distance,Region,Age,Purchased Bike
0,12496,Married,Female,40000,1,Bachelors,Skilled Manual,Yes,0,0-1 Miles,Europe,42,No
1,24107,Married,Male,30000,3,Partial College,Clerical,Yes,1,0-1 Miles,Europe,43,No
2,14177,Married,Male,80000,5,Partial College,Professional,No,2,2-5 Miles,Europe,60,No
3,24381,Single,Male,70000,0,Bachelors,Professional,Yes,1,5-10 Miles,Pacific,41,Yes
4,25597,Single,Male,30000,0,Bachelors,Clerical,No,0,0-1 Miles,Europe,36,Yes


In [69]:
# смотрим размер датафрейма
data.shape

(1000, 13)

Выведите статистики по категориальным признакам, чтобы посмотреть, сколько категорий в каждом категориальном (нечисловом) признаке.

Для этого можно воспользоваться методом `describe` из библиотеки pandas со значением параметра `include = 'object'`.

In [70]:
data.describe(include = 'object')

,Marital Status,Gender,Education,Occupation,Home Owner,Commute Distance,Region,Purchased Bike
count,1000,1000,1000,1000,1000,1000,1000,1000
unique,2,2,5,5,2,5,3,2
top,Married,Male,Bachelors,Professional,Yes,0-1 Miles,North America,No
freq,539,509,306,276,685,366,508,519


**Вопрос:** в каком категориальном признаке встречаются три различных значения?

Region

Закодируйте все категориальные столбцы с двумя категориями следующим образом:  
самая часто встречающаяся категория превращается в 1, другая в 0.

In [71]:
def binary_encode(series):

    most_common_category = series.value_counts().idxmax()
    return (series == most_common_category).astype(int)

for column in data.columns:
    unique_values = data[column].nunique()
    if unique_values == 2:  # Проверяем, что в столбце только две категории
        data[column] = binary_encode(data[column])


In [72]:
data=data.drop(columns=['Occupation','Commute Distance','Region','Education'])

Удалите остальные категориальные столбцы.

**Вопрос:** сколько категориальных столбцов вы удалили?

4

Удалите столбец `ID`, так как он по сути нечисловой.

In [73]:
data=data.drop(columns=['ID'])

Сформируйте матрицу объект-признак `X` и вектор `y` с целевой переменной.  
Целевая переменная - это последний столбец, `Purchased Bike`.

In [74]:
X=data.drop(columns='Purchased Bike')
y=data['Purchased Bike']

Разбейте данные на тренировочную и тестовую часть (`Xtrain`, `Xtest`, `ytrain`, `ytest`), в тест отправьте 30% данных.  
Зафиксируйте `random_state = 42`.

In [75]:
from sklearn.model_selection import train_test_split

Xtrain, Xtest, ytrain, ytest = train_test_split(X,y,test_size=0.3,random_state = 42)

**Вопрос:** сколько объектов в матрице `Xtrain`?

In [76]:
Xtrain.shape

(700, 7)

Почти всё готово для обучения модели!

Осталось отмасштабировать матрицу `X`, так как линейные модели чувствительны к масштабу данных.

*  Обучите на тренировочной матрице (`Xtrain`) `MinMaxScaler` из библиотеки `sklearn.preprocessing`
*  Примените масштабирование и к `Xtrain`, и к `Xtest`
*  Переведите полученные после масштабирования `np.array` обратно в pandas `dataframe`.

Полученные масштабированные матрицы назовите, как и раньше, `Xtrain` и `Xtest`.

In [77]:
from sklearn.preprocessing import MinMaxScaler

min_max=MinMaxScaler()
X_train_scaled=min_max.fit_transform(Xtrain)
X_test_size=min_max.transform(Xtest)

Xtrain=pd.DataFrame(X_train_scaled, columns=Xtrain.columns)
Xtest=pd.DataFrame(X_test_size, columns=Xtest.columns)

Теперь обучите логистическую регрессию на тренировочных данных

In [78]:
from sklearn.linear_model import LogisticRegression

model=LogisticRegression()
model.fit(Xtrain,ytrain)


LogisticRegression()

Сделайте предсказания на тренировочных и на тестовых данных.

In [79]:
y_pred_test=model.predict(Xtest)
y_pred_train=model.predict(Xtrain)

Оцените значение accuracy на трейне и на тесте.

In [80]:
from sklearn.metrics import accuracy_score

print(accuracy_score(y_pred_test,ytest))
print(accuracy_score(y_pred_train,ytrain))

0.58
0.6342857142857142


Качество модели получилось невысоким, зато модель не переобучена.

Попробуем добавить новых признаков в модель, используя [PolynomialFeatures](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html).

Создайте полиномиальные признаки degree = 2.

Как обычно:
*  `fit` делайте на тренировочных данных
*  `transform` и на тренировочных, и на тестовых данных. Затем верните результат к формату pandas `dataframe`.

Полученные матрицы назовите, как и раньше, `Xtrain` и `Xtest`.

In [81]:
from sklearn.preprocessing import PolynomialFeatures

poly=PolynomialFeatures(degree=2)
poly.fit(Xtrain)

X_train_pol=poly.transform(Xtrain)
X_test_pol=poly.transform(Xtest)

Xtrain=pd.DataFrame(X_train_pol)
Xtest=pd.DataFrame(X_test_pol)

**Вопрос:** на сколько признаков стало больше при добавлении полиномиальных признаков второй степени?

In [82]:
Xtrain.shape

(700, 36)

Заново обучите логистическую регрессию, уже на расширенной матрице признаков, и сделайте предсказания на трейне и тесте, а затем оцените качество (*accuracy*).

In [83]:
model=LogisticRegression()
model.fit(Xtrain,ytrain)

y_pred_test=model.predict(Xtest)
y_pred_train=model.predict(Xtrain)

print(accuracy_score(y_pred_test,ytest))
print(accuracy_score(y_pred_train,ytrain))

0.6233333333333333
0.6871428571428572


**Вопрос:** на сколько повысилось качество модели на тестовых данных?  
Ответ округлите до сотых.

Появились новые требования от заказчика!

Заказчик просит, чтобы:
*  доля найденных моделью потенциальных покупателей была максимальной
*  accuracy при этом была не ниже, чем 0.6 (отклонения *accuracy* на тестовых данных на $\pm 0.05$ допустимы).

Сначала посмотрите, какие значения *recall* и *accuracy* имеют предсказния модели на тесте с классами, предсказанными по умолчанию (методом `predict`).

In [84]:
from sklearn.metrics import recall_score
print(recall_score(y_pred_test,ytest))
print(accuracy_score(y_pred_test,ytest))

0.6023391812865497
0.6233333333333333


Подберём порог для перевода вероятностей в классы, чтобы оптимизировать требуемые метрики!

Разобъем тренировочные данные на трейн и валидацию, чтобы по валидационной части подбирать порог.

In [85]:
XtrainS, Xval, ytrainS, yval = train_test_split(Xtrain, ytrain, test_size=0.3, random_state=42)

XtrainS.shape, Xval.shape

((490, 36), (210, 36))

* Обучите модель на тренировочных данных.
* Предскажите вероятности положительного класса на валидационных данных

В цикле для каждого значения порога:
*  переведите вероятности в классы
*  вычислите полноту (на валидационных данных)

Выведите на экран:

1) значение порога, дающее максимальный *recall*, при условии, что *accuracy* $\geq$ 0.6.

2) значение *recall* при этом пороге

3) значение *accuracy* для этого порога


Ищите порог на отрезке от 0 до 1 с шагом 0.01.

In [110]:
model=LogisticRegression()
model.fit(Xtrain,ytrain)
probab=model.predict_proba(Xval)

best_recall = 0
best_threshold = 0
best_accuracy = 0

for i in np.arange(0, 1, 0.01):
    pred=(probab[:,1]>i).astype(int)
    
    rec = recall_score(yval, pred)
    ac = accuracy_score(yval, pred)

    if ac >= 0.6 and rec > best_recall:
        best_recall = rec
        best_threshold = i
        best_accuracy = ac

    

Теперь заново обучите модель на исходных тренировочных данных (`Xtrain`, `ytrain`), предскажите вероятности на тесте и переведите их в классы по найденному порогу.

In [115]:
model=LogisticRegression()
model.fit(Xtrain,ytrain)

prob=model.predict_proba(Xtest)
pred=(prob[:,1]>best_threshold).astype(int)

rec = recall_score(ytest, pred)
ac = accuracy_score(ytest, pred)

rec,ac

(0.8513513513513513, 0.5333333333333333)

**Вопрос:** какое значение *recall* получилось на тестовых данных после подбора порога?  
Ответ округлите до десятых.

При помощи подбора порога удалось сильно увеличить значение *recall*!  
Однако, как видно, на тесте не удалось сохранить условие $accuracy \geq 0.6$ (но в допустимые рамки уложились!)

Это свидетельство небольшого переобучения модели. Однако в сухом остатке имеет значительное увеличение полноты, что является приоритетом для заказчика.